In [1]:
import typing
import torch
from torch import nn, Tensor
from dataclasses import dataclass
from conf import *
import torch.nn.functional as F
from avalanche.benchmarks.classic.cfashion_mnist import SplitFMNIST
import torchvision.transforms as transforms
from experiment.experiment import Experiment, BaseHyperParameters
from network.coders import PackNetDenseEncoder, PackNetDenseDecoder, PackNetDenseHead

%matplotlib auto
%load_ext autoreload
%autoreload 2
from network.deep_generative import DVAE, DVAE_Loss


Using matplotlib backend: <object object at 0x7f4214f9ccf0>


In [2]:



@dataclass
class HyperParams(BaseHyperParameters):
    pass


class MyExperiment(Experiment):

    hp: HyperParams
    network: DVAE

    classifier_weight: float = 50.0

    def __init__(self, hp: HyperParams) -> None:
        super().__init__(hp)

        self.after_eval_forward = self.after_forward

    def make_network(self) -> nn.Module:

        latent_dims = 64

        encoder = PackNetDenseEncoder((1, 28, 28), 64, [512, 256, 128])
        decoder = PackNetDenseDecoder((1, 28, 28), 64, [128, 256, 512])

        head = PackNetDenseHead(latent_dims, 10)
        fc_mu = PackNetDenseHead(latent_dims, latent_dims)
        fc_var = PackNetDenseHead(latent_dims, latent_dims)

        return DVAE(latent_dims, encoder, decoder, head, fc_mu, fc_var)

    def make_optimizer(self, parameters) -> torch.optim.Optimizer:
        optimizer = torch.optim.Adam(parameters, self.hp.lr)
        return optimizer

    def make_criterion(self):
        self.criterion = DVAE_Loss(1.0, 1.0, 0.0)
        def VAE_criterion(output, target):
            return self.criterion.loss(self.last_mb_output, target)
        return VAE_criterion

    def make_scenario(self):
        transform = transforms.Compose(
            [transforms.ToTensor(), transforms.Normalize((0.2860,), (0.3530,))]
        )
        scenario = SplitFMNIST(
            n_experiences=5,
            fixed_class_order=[0, 1, 2, 3, 4, 5, 6, 7, 8, 9],
            dataset_root=DATASETS,
            train_transform=transform,
            eval_transform=transform)
        return scenario


In [5]:
experiment = MyExperiment(
    HyperParams(
        lr=0.001,
        train_mb_size=32,
        train_epochs=50,
        eval_mb_size=128,
        eval_every=-1,
        device="cuda"
    )
).train()


Network: <class 'network.deep_generative.DVAE'>
 > Has the <class 'network.trait.AutoEncoder'> trait
 > Has the <class 'network.trait.Classifier'> trait
 > Has the <class 'network.trait.Samplable'> trait
Start of experience: 0
Current Classes:     [0, 1]
Experience size:     12000
Start of experience: 1
Current Classes:     [2, 3]
Experience size:     12000


KeyboardInterrupt: 